### Import

In [21]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

### Load data

In [25]:
def load_data():
    # this_directory = os.path.dirname(os.path.abspath(__file__))
    data = pd.read_csv(os.path.join('hn', 'HN_radiomicFeatures.csv'), index_col=0)

    return data


data = load_data()
label = data["label"]
data = data.drop(["label"], axis=1)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 113
The number of columns: 159


### Check for missing data

In [26]:
# Detect missing values in dataframe
data.isnull().values.any()

False

### Data split

In [27]:
train_val_data, test_data, train_val_label, test_label = train_test_split(data, label, train_size=0.8)
train_data, val_data, train_label, val_label = train_test_split(train_val_data, train_val_label, train_size=0.85)

print(f'The number of train samples: {train_data.shape[0]}')
print(f'The number of validation samples: {val_data.shape[0]}')
print(f'The number of test samples: {test_data.shape[0]}')

The number of train samples: 76
The number of validation samples: 14
The number of test samples: 23


In [29]:
scaler = StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)

clf = SGDClassifier()
clf.fit(train_data, train_label)
print(clf.coef_)

[[ 28.37876301  -4.90326392 -34.72880681  24.34176813  -6.16553379
  -12.26809404   5.87237424  30.53260185 -45.85669779   5.14895686
   77.06325688  10.59764544 -31.19730653 -24.23879797 -10.16815352
   -8.85792536   2.32289558 -34.71616417   4.54575441  21.26538674
   28.16910033 -28.38139578  -5.36068437  50.79613537 -30.82888896
    1.42330644  16.58706671 -47.34396819   7.68426946   0.57065668
  -40.84352467  25.25472581  23.15036843  13.87793949 -13.87793949
   -2.46938103  26.33942742  24.2954847   16.88157481   2.46938103
   37.67923894  19.26654308 -47.65002346 -48.38780981  25.8174952
  -16.16301211 -14.68214722  -4.55277078  -1.65893346  -1.3560661
    1.3560661    3.90079883  27.14445578  27.12420597  22.85226393
   -3.90079883  26.60942553  20.11667007  10.86777947  13.33588688
   10.23986751 -21.53282686 -40.56329477  27.50644014  -0.12859415
    4.37768281  -4.14708368  -9.0607616  -49.02368493   1.72119431
  -32.17278301   8.86207054 -15.58956318  -6.10255664 -30.255304